In [108]:
'''
    moduls used in this prooject is 
        ---> numpy , pandas , sklearn
        ---> streamlit , request , os, ast

'''
import ast
import numpy as np
import pandas as pd
import requests
import streamlit as st 
import os 
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.text import TokenSearcher
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [109]:
'''
    importing datasets using pandas in csv form to pandas dataFrame
    ---> 01. credits 
    ---> 02. movies
    ---> 03. keywords
    
'''

credits = pd.read_csv('dataset/movies_5000_data/tmdb_5000_credits.csv')
movies = pd.read_csv('dataset/movies_5000_data/tmdb_5000_movies.csv')
credits = credits.rename(columns={'movie_id':'id'})
credits= credits[:1000]
movies = movies[:1000]
credits.dropna()
movies.dropna()
print(movies.shape)
print(credits.shape)


(999, 20)
(999, 4)


In [110]:
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,50000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",http://gamerthemovie.com,18501,"[{""id"": 4565, ""name"": ""dystopia""}, {""id"": 9678...",en,Gamer,Mind-control technology has taken society by a...,27.631262,"[{""name"": ""Lakeshore Entertainment"", ""id"": 126...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-09-03,40828540,95,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"In the near future, you don't live to play... ...",Gamer,5.6,760
995,60000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 18, ""na...",NaN,109491,"[{""id"": 393, ""name"": ""civil war""}, {""id"": 531,...",en,Beautiful Creatures,Ethan Wate just wants to get to know Lena Duch...,40.721373,"[{""name"": ""Alcon Entertainment"", ""id"": 1088}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-02-13,60052138,124,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Dark secrets will come to light.,Beautiful Creatures,5.6,984
996,55000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 80, ""nam...",NaN,9275,"[

In [111]:
credits = credits[['id','title','cast','crew']]

In [112]:
movies = movies.dropna()
credits = credits.dropna()

In [113]:
''' 
        01. useful column from movies dataFrame is --> 
                {
                        adult,genres,id,title,overview,poster_path,imdb_id
                }
        02. merging the credits and keywords dataFrame with movies on the basis of id column

'''

movies = movies.merge(credits,on='id')
movies = movies[['title_x','id','genres','overview','cast','crew','keywords']]

In [115]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      999 non-null    int64 
 1   title   999 non-null    object
 2   cast    999 non-null    object
 3   crew    999 non-null    object
dtypes: int64(1), object(3)
memory usage: 31.3+ KB


In [117]:
'''   
    01. Preprocessing of column cast
    02. Preprocessing of column crew
    03. Preprocessing of column keywords
    04. Preprocessing of column overview
    05. Preprocessing of column genres

'''
def genre_column_transform(obj):
    genre_list = []
    for ls in ast.literal_eval(obj):
        genre_list.append(ls['name'])
    return genre_list

def keywords_column_transformation(obj):
    keyword_tag = []
    for ls in ast.literal_eval(obj):
        keyword_tag.append(ls['name'])
    return keyword_tag

def finding_director_name(obj):
    name = ''
    for ls in ast.literal_eval(obj):
        if ls['job']=='Director':
            name = ls['name']
    return name

def character_in_movies(obj):
    list_hero  = []
    x = 0
    for ls in ast.literal_eval(obj):
        if x==4:
            break
        list_hero.append(ls['name'])
        x += 1
    return list_hero

'''    
        converting list into string so that we can 
        create bag of words and further convert into
        vector object for finding similarity between
        two movies using angle between two vectors

'''
def transform_str(obj):
    return " ".join(obj)

def transform_to_str(obj):
    lis = []
    for ls in obj:
        ls = ls.replace(" ","")
        lis.append(ls)
    return " ".join(lis)

def transform_to_str1(obj):
    obj = obj.replace(" ","")
    return obj

'''    
        merging genres ,keywords and cast and crew together
        to form tags column which will be vectorizable

'''
def merge_cols(df,f1,f2,f3,f4,f5):
    df['tags'] = f1+f2+f3+f4
    return df


In [118]:
'''
    Data wrangling and preprocessing 
    for converting data into our domain 
    specific format to find similarity between 
    movies.
'''
#'title_x','id','genres','overview','cast','crew','keywords'

movies['genres'] = movies['genres'].apply(genre_column_transform)
movies['keywords'] =movies['keywords'].apply(keywords_column_transformation)
movies['crew'] = movies['crew'].apply(finding_director_name)
movies['cast'] = movies['cast'].apply(character_in_movies)
movies['cast'] = movies['cast'].apply(transform_to_str)
movies['crew'] =   movies['crew'].apply(transform_to_str1)[0]
movies['keywords'] = movies['keywords'].apply(transform_to_str)
movies['genres'] = movies['genres'].apply(transform_to_str)
movies = merge_cols(movies,movies['overview'],movies['genres'],movies['keywords'],movies['cast'],movies['crew'])
movies = movies[['id','title_x','tags']]
movies = movies[:1000]

In [119]:
movies = movies.dropna()

In [120]:
def stemming(text_data):
    lis = []
    porter = PorterStemmer()
    for ls in text_data.split(" "):
        stemmed_text = porter.stem(ls)
        lis.append(stemmed_text)
    return " ".join(lis)

In [121]:
'''   
    stemming and vectorization of textual data and converting it
    into cosine similarity matrics

'''

cv = CountVectorizer(max_features=1500,stop_words='english')
movies['tags'] = movies['tags'].apply(stemming)
text_vectors = cv.fit_transform(movies['tags']).toarray()
matric_of_similarity =  cosine_similarity(text_vectors)

In [122]:
def movie_recommendation(movies,movie_name,cosine_similarity):
    cnt = 0
    movie_index = movies[movies['title_x']==movie_name].index[0]
    cosine_similarity_theta = cosine_similarity[movie_index]
    recommended_movie_list = []
    ls = sorted(list(enumerate(cosine_similarity_theta)),reverse=True,key=lambda x:x[1])
    for x in ls:
        if cnt==6:
            break
        cnt+= 1
        recommended_movie_list.append(x[0])
    return recommended_movie_list[1:]

def fetch_movie_name_with_index(movies,movie_list):
    list_of_info = []
    for x in movie_list:
        ls = []
        ls.append(movies.iloc[x]['id'])
        ls.append(movies.iloc[x]['title_x'])
        list_of_info.append(ls)
    return list_of_info

In [123]:
# matric_of_similarity[0]
ls = movie_recommendation(movies,'Avatar',matric_of_similarity)
ls

[331, 423, 61, 118, 329]

In [124]:
info = fetch_movie_name_with_index(movies,ls)
print(info)

[[44943, 'Battle: Los Angeles'], [11260, 'Meet Dave'], [137113, 'Edge of Tomorrow'], [2310, 'Beowulf'], [395, 'AVP: Alien vs. Predator']]


In [125]:
response = requests.get(url='https://api.themoviedb.org/3/movie/157336?api_key=781a96197d3ef620e98a05bc9bf85cbb&language=en-US').json()

In [126]:
path = "https://image.tmdb.org/t/p/w185"+response['poster_path']

In [127]:
pickle.dump(movies,open('movie_list.pkl','wb'))
pickle.dump(matric_of_similarity,open('similarity.pkl','wb'))